In [1]:
from collections import Counter

import numpy
import pygeoprocessing
import pandas
import ipywidgets as widgets
from IPython.display import display

In [2]:
# ! mamba install jupyterlab_widgets

In [2]:
raster_path = 'H:/Shared drives/Online Urban ES Modeling Tool/Data from Chris/Combined NLCD NLUD Tree/overlay.tif'
csv_path = 'H:/Shared drives/Online Urban ES Modeling Tool/Data from Chris/Combined NLCD NLUD Tree/combined_structure.csv'

In [3]:
df = pandas.read_csv(csv_path)

In [4]:
df.head()

,lucode,code,nlud,nlud tier 1,nlud tier 2,nlud tier 3,nlud colors,nlcd,nlcd lulc,nlcd colors,tree,tree canopy cover,tree colors
0,0,111110,111,Water,Natural - area,Lake,#0070ff,11,Open Water,#476BA0,0,none,#ffffff
1,1,111111,111,Water,Natural - area,Lake,#0070ff,11,Open Water,#476BA0,1,low,#e5f5e0
2,2,111112,111,Water,Natural - area,Lake,#0070ff,11,Open Water,#476BA0,2,medium,#a1d99b
3,3,111113,111,Water,Natural - area,Lake,#0070ff,11,Open Water,#476BA0,3,high,#31a354
4,4,111120,111,Water,Natural - area,Lake,#0070ff,12,Perennial Ice/Snow,#D1DDF9,0,none,#ffffff


In [5]:
def count(c, block):
    return c + Counter(block)
counts = pygeoprocessing.raster_reduce(count, (raster_path, 1), Counter())

In [6]:
def unique(unique, block):
    return unique.union(set(numpy.unique(block)))
value_set = pygeoprocessing.raster_reduce(unique, (raster_path, 1), set())

In [7]:
len(value_set)

1556

In [8]:
table = df.loc[df.lucode.isin(value_set)]

In [9]:
table.head()

,lucode,code,nlud,nlud tier 1,nlud tier 2,nlud tier 3,nlud colors,nlcd,nlcd lulc,nlcd colors,tree,tree canopy cover,tree colors
0,0,111110,111,Water,Natural - area,Lake,#0070ff,11,Open Water,#476BA0,0,none,#ffffff
1,1,111111,111,Water,Natural - area,Lake,#0070ff,11,Open Water,#476BA0,1,low,#e5f5e0
2,2,111112,111,Water,Natural - area,Lake,#0070ff,11,Open Water,#476BA0,2,medium,#a1d99b
8,8,111210,111,Water,Natural - area,Lake,#0070ff,21,"Developed, Open Space",#DDC9C9,0,none,#ffffff
9,9,111211,111,Water,Natural - area,Lake,#0070ff,21,"Developed, Open Space",#DDC9C9,1,low,#e5f5e0


In [14]:
table.columns = [col.replace(' ', '_') for col in table.columns]

In [15]:
table.columns

Index(['lucode', 'code', 'nlud', 'nlud_tier_1', 'nlud_tier_2', 'nlud_tier_3',
       'nlud_colors', 'nlcd', 'nlcd_lulc', 'nlcd_colors', 'tree',
       'tree_canopy_cover', 'tree_colors'],
      dtype='object')

In [16]:
table.to_csv('H:/Shared drives/Online Urban ES Modeling Tool/Data from Chris/Combined NLCD NLUD Tree/present_in_raster.csv', index=False)

In [36]:
nlcd_options = list(set([(name, value) for name, value in zip(table['nlcd lulc'], table['nlcd'])]))
nlcd = widgets.Dropdown(
    options=nlcd_options
)
display(nlcd)

Dropdown(options=(('Open Water', 11), ('Developed, High Intensity', 24), ('Cultivated Crops', 82), ('Woody Wet…

In [63]:
t = table.loc[table.nlcd == nlcd.value]
nlud_options = list(set([(' | '.join([name1, name2, name3]), value) for name1, name2, name3, value in zip(t['nlud tier 1'], t['nlud tier 2'], t['nlud tier 3'] , t['nlud'])]))
nlud = widgets.Dropdown(
    options=nlud_options
)
display(nlud)

Dropdown(options=(('Conservation | Private easement | Wildlife conservation', 531), ('Water | Natural - area |…

In [66]:
table.loc[(table.nlcd == nlcd.value) & (table.nlud == nlud.value)]

,lucode,code,nlud,nlud tier 1,nlud tier 2,nlud tier 3,nlud colors,nlcd,nlcd lulc,nlcd colors,tree,tree canopy cover,tree colors
1440,1440,221110,221,Built-up,Commercial,Office,#df73ff,11,Open Water,#476BA0,0,none,#ffffff
1441,1441,221111,221,Built-up,Commercial,Office,#df73ff,11,Open Water,#476BA0,1,low,#e5f5e0
1442,1442,221112,221,Built-up,Commercial,Office,#df73ff,11,Open Water,#476BA0,2,medium,#a1d99b


In [67]:
counts[1440]

14

In [18]:
counts > 1000

TypeError: '>' not supported between instances of 'Counter' and 'int'